Extending spark.graphx.lib.ShortestPaths to GraphXShortestWeightedPaths
=======================================================================

### 2016-2020, Ivan Sadikov and Raazesh Sainudiin

We extend Shortest Paths algorithm in Spark's GraphX Library to allow
for user-specified edge-weights as an edge attribute.

This is part of *Project MEP: Meme Evolution Programme* and supported by
databricks academic partners program.

The analysis is available in the following databricks notebook: \*
<http://lamastex.org/lmse/mep/src/GraphXShortestWeightedPaths.html>

\`\`\` Copyright 2016 Ivan Sadikov and Raazesh Sainudiin

Licensed under the Apache License, Version 2.0 (the "License"); you may
not use this file except in compliance with the License. You may obtain
a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License. \`\`\`

### Let's modify shortest paths algorithm to allow for user-specified edge-weights

Update shortest paths algorithm to work over edge attribute of
edge-weights as Double, key concepts are: - we increment map with delta,
which is `edge.attr` - edge attribute is anything numeric, tested on
Double - infinity value is not infinity, but `Integer.MAX_VALUE`

Modifying the following code: \*
https://github.com/apache/spark/blob/master/graphx/src/main/scala/org/apache/spark/graphx/lib/ShortestPaths.scala

Explained here: \*
http://note.yuhc.me/2015/03/graphx-pregel-shortest-path/

In [ ]:
import scala.reflect.ClassTag
import org.apache.spark.graphx._

/**
 * Computes shortest weighted paths to the given set of landmark vertices, returning a graph where each
 * vertex attribute is a map containing the shortest-path distance to each reachable landmark.
 * Currently supports only Graph of [VD, Double], where VD is an arbitrary vertex type.
 *
 * The object also include a function which transforms the resulting graph into a path_graph between a 
 * specific starting node and goal node. Each edge in the path_grpah is either 1 or 0 depending if it is 
 * the shortest path or not.
 *
 */
object ShortestPath extends Serializable {

  // When finding the shortest path each node stores a map from the itself to each goal node.
  // The map returns an array includeing the total distance to the goal node as well as the
  // next node pn the shortest path to the goal node. The last value in the array is only 
  // populated with the nodes own id and is only used for computational convenience. 
  type SPMap = Map[VertexId, Tuple3[Double, VertexId, VertexId]]
  
  // PN holds the information of the path nodes which are used for creating a path graph
  // PN = ('Distance left to goal node', 'Next path node id', 'Goal node', 'Is on path')
  type PN = Tuple4[Double, VertexId, VertexId, Boolean] 
  
  private val INITIAL_DIST = 0.0
  private val DEFAULT_ID = -1L
  private val INFINITY = Int.MaxValue.toDouble

  private def makeMap(x: (VertexId, Tuple3[Double, VertexId, VertexId])*) = Map(x: _*)
  
  private def incrementMap(spmap: SPMap, delta: Double, id: VertexId): SPMap = { 
    spmap.map { case (v, d) => v -> (Tuple3(d._1 + delta, d._3, id)) }
  }

  private def addMaps(spmap1: SPMap, spmap2: SPMap): SPMap = {
    (spmap1.keySet ++ spmap2.keySet).map {
    k =>{
        if (spmap1.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._1 < spmap2.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._1) 
                k -> (Tuple3(spmap1.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._1, 
                             spmap1.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._2, 
                             spmap1.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._3))
        else 
                k -> (Tuple3(spmap2.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._1, 
                             spmap2.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._2, 
                             spmap2.getOrElse(k, Tuple3(INFINITY, DEFAULT_ID, DEFAULT_ID))._3))
        }
    }.toMap
  }
  
  // at this point it does not really matter what vertex type is
  def run[VD](graph: Graph[VD, Double], landmarks: Seq[VertexId]): Graph[SPMap, Double] = {
    val spGraph = graph.mapVertices { (vid, attr) =>
      // initial value for itself is 0.0 as Double
      if (landmarks.contains(vid)) makeMap(vid -> Tuple3(INITIAL_DIST, DEFAULT_ID, DEFAULT_ID)) else makeMap()
    }

    val initMaps = makeMap()

    def vProg(id: VertexId, attr: SPMap, msg: SPMap): SPMap = {
      addMaps(attr, msg)
    }

    def sendMsg(edge: EdgeTriplet[SPMap, Double]): Iterator[(VertexId, SPMap)] = {
      val newAttr = incrementMap(edge.dstAttr, edge.attr, edge.srcId)
      if (edge.srcAttr != addMaps(newAttr, edge.srcAttr)) Iterator((edge.srcId, newAttr))
      else Iterator.empty
    }

    Pregel(spGraph, initMaps)(vProg, sendMsg, addMaps)
  }
  
  def create_path_graph[VD](graph: Graph[SPMap, Double], goalId: VertexId, startId: VertexId): Graph[PN, Int] = {
    // For a given goal node we remove the lookup map and extend the state to a Tuple5 with the goal id and a boolean
    val path = graph.mapEdges(e => 0)
              .mapVertices((vertixId, attr) => {
                if (attr.contains(goalId)) {
                  val path_step = attr(goalId)
                  if (vertixId == path_step._3 && path_step._2 == -1L)
                    (path_step._1, goalId, goalId, false) // while we are at it, we clean up the state a bit
                  else  
                    (path_step._1, path_step._2, goalId, false)
                } else// If the vertice does not have a map to our goal we add a default value to it
                    (INFINITY, -1L, -1L, false)
              })

      def mergeMsg(msg1: VertexId, msg2: VertexId): VertexId = { // we should only get one msg
          msg2
      }

      def vprog(id: VertexId, attr: PN, msg: VertexId): PN = {
        // Check that the current node is the one adressed in the message
        if (id == msg)
          (attr._1, attr._2, attr._3, true)
        else // If the message is not addressed to the current node (happens for inital message), use the old value 
          attr
      }
      def sendMsg(triplet: EdgeTriplet[PN, Int]): Iterator[(VertexId, VertexId)] = {
        // If dstId is the next node on the path and has not yet been activated
        if (triplet.srcAttr._2 == triplet.dstId && triplet.srcAttr._4 && !triplet.dstAttr._4) 
          Iterator((triplet.dstId, triplet.dstId))// Send next msg
        else
          Iterator.empty// Do nothing
      }

      Pregel(path, startId)(vprog, sendMsg, mergeMsg).mapTriplets(triplet => {
        if(triplet.srcAttr._2 == triplet.dstId && triplet.srcAttr._4)
          1
        else
          0
      })
  }
}

println("Usage: val result = GraphXShortestWeightedPaths.run(graph, Seq(4L, 0L, 9L))")

  

>     Usage: val result = GraphXShortestWeightedPaths.run(graph, Seq(4L, 0L, 9L))
>     import scala.reflect.ClassTag
>     import org.apache.spark.graphx._
>     defined object ShortestPath

  

### Generate test graph

Generate simple graph with double weights for edges

In [ ]:
import scala.util.Random

import org.apache.spark.graphx.{Graph, VertexId}
import org.apache.spark.graphx.util.GraphGenerators

// A graph with edge attributes containing distances
val graph: Graph[Long, Double] = GraphGenerators.logNormalGraph(sc, numVertices = 150, seed=123L).mapEdges { e => 
  // to make things nicer we assign 0 distance to itself
  if (e.srcId == e.dstId) 0.0 else Random.nextDouble()
}





  

>     import scala.util.Random
>     import org.apache.spark.graphx.{Graph, VertexId}
>     import org.apache.spark.graphx.util.GraphGenerators
>     graph: org.apache.spark.graphx.Graph[Long,Double] = org.apache.spark.graphx.impl.GraphImpl@7d63ae0f

In [ ]:
// Create an RDD for the vertices
val nodes: RDD[(VertexId, String)] =
  sc.parallelize(Seq((0L, "0"), 
                     (1L, "1"), 
                     (2L, "2"), 
                     (3L, "3"), 
                     (4L, "4"), 
                     (5L, "5"), 
                     (6L, "6"), 
                     (7L, "7")
                    )
                )


// Create an RDD for edges
val connections: RDD[Edge[Double]] =
  sc.parallelize(Seq(Edge(0L, 1L, 1),   
                     Edge(1L, 2L, 1),
                     Edge(2L, 3L, 1), 
                     Edge(3L, 4L, 1),
                     Edge(0L, 4L, 100),
                     Edge(5L, 6L, 1),
                     Edge(6L, 7L, 1),
                     Edge(7L, 5L, 1),
                     Edge(4L, 2L, 1),
                     Edge(2L, 3L, 1)
                    )
                )
// Build the initial Graph
val default_node = "default"
val graph = Graph(nodes, connections, default_node)



  

>     nodes: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, String)] = ParallelCollectionRDD[18056] at parallelize at command-1884488408458313:3
>     connections: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Double]] = ParallelCollectionRDD[18057] at parallelize at command-1884488408458313:17
>     default_node: String = default
>     graph: org.apache.spark.graphx.Graph[String,Double] = org.apache.spark.graphx.impl.GraphImpl@6901db86

In [ ]:
graph.vertices.take(150).foreach(println)
graph.edges.take(150).foreach(println)

  

>     (0,0)
>     (1,1)
>     (2,2)
>     (3,3)
>     (4,4)
>     (5,5)
>     (6,6)
>     (7,7)
>     Edge(0,1,1.0)
>     Edge(1,2,1.0)
>     Edge(2,3,1.0)
>     Edge(3,4,1.0)
>     Edge(0,4,100.0)
>     Edge(5,6,1.0)
>     Edge(6,7,1.0)
>     Edge(7,5,1.0)
>     Edge(3,2,1.0)
>     Edge(2,3,1.0)

In [ ]:
val landMarkVertexIds = Seq(4L)
val result = ShortestPath.run(graph, landMarkVertexIds)
// Found shortest paths
println(result.vertices.collect.mkString("\n"))

  

>     (0,Map(4 -> (4.0,1,0)))
>     (1,Map(4 -> (3.0,2,1)))
>     (2,Map(4 -> (2.0,3,2)))
>     (3,Map(4 -> (1.0,-1,3)))
>     (4,Map(4 -> (0.0,-1,-1)))
>     (5,Map())
>     (6,Map())
>     (7,Map())
>     landMarkVertexIds: Seq[Long] = List(4)
>     result: org.apache.spark.graphx.Graph[ShortestPath.SPMap,Double] = org.apache.spark.graphx.impl.GraphImpl@139aeb54

In [ ]:
 val path_graph = ShortestPath.create_path_graph(result, 4L, 2L)
println(path_graph.vertices.collect.mkString("\n"))
println(path_graph.edges.collect.mkString("\n"))

  

>     (0,(4.0,1,4,false))
>     (1,(3.0,2,4,false))
>     (2,(2.0,3,4,true))
>     (3,(1.0,4,4,true))
>     (4,(0.0,-1,4,true))
>     (5,(2.147483647E9,-1,-1,false))
>     (6,(2.147483647E9,-1,-1,false))
>     (7,(2.147483647E9,-1,-1,false))
>     Edge(0,1,0)
>     Edge(1,2,0)
>     Edge(2,3,1)
>     Edge(3,4,1)
>     Edge(0,4,0)
>     Edge(5,6,0)
>     Edge(6,7,0)
>     Edge(7,5,0)
>     Edge(4,2,0)
>     Edge(2,3,1)
>     path_graph: org.apache.spark.graphx.Graph[ShortestPath.PN,Int] = org.apache.spark.graphx.impl.GraphImpl@f2499ed

In [ ]:
display(path_graph.edges.toDF)

  

[TABLE]

In [ ]:
val startId = 5L
val goalId = 4L
val INFINITY = Int.MaxValue.toDouble
// set all edges to zero
// For a given goal node we remove the lookup map and extend the state to a Tuple5 with the goal id and a boolean
val path = result.mapEdges(e => 0)
                 .mapVertices((vertixId, attr) => {
                   if (attr.contains(goalId)) {
                     val path_step = attr(goalId)
                     if (vertixId == path_step._3 && path_step._2 == -1L)
                       (path_step._1, goalId, goalId, false) // while we are at it, we clean up the state a bit
                     else  
                       (path_step._1, path_step._2, goalId, false)
                   } else// If the vertice does not have a map to our goal we add a default value to it
                       (INFINITY, -1L, -1L, false)
                 })
println(path.vertices.collect.mkString("\n"))
// PN = ('Distance to goal node', 'Next path node id', 'Goal node', 'Is on path')
type PN = Tuple4[Double, VertexId, VertexId, Boolean] 

val initMsg = startId

def mergeMsg(msg1: VertexId, msg2: VertexId): VertexId = { // we should only get one msg
    msg1
}

def vprog(id: VertexId, attr: PN, msg: VertexId): PN = {
  // Check that the current node is the one adressed in the message
  if (id == msg)
    (attr._1, attr._2, attr._3, true)
  else // If the message is not addressed to the current node (happens for inital message), use the old value 
    (attr._1, attr._2, attr._3, attr._4)
}
def sendMsg(triplet: EdgeTriplet[PN, Int]): Iterator[(VertexId, VertexId)] = {
  // If dstId is the next node on the path and has not yet been activated
  if (triplet.srcAttr._2 == triplet.dstId & triplet.srcAttr._4 == true & triplet.dstAttr._4 != true) 
    Iterator((triplet.dstId, triplet.dstId))// Send next msg
  else
    Iterator.empty// Do nothing
}

val result2 = Pregel(path, initMsg)(vprog, sendMsg, mergeMsg)
println(result2.vertices.collect.mkString("\n"))

val finalResult = result2.mapTriplets(triplet => {
  if(triplet.srcAttr._2 == triplet.dstId && triplet.srcAttr._4 == true && triplet.dstAttr._4 == true)
    1
  else
    0
})

println(finalResult.edges.collect.mkString("\n"))



  

>     (0,(4.0,1,4,false))
>     (1,(3.0,2,4,false))
>     (2,(2.0,3,4,false))
>     (3,(1.0,4,4,false))
>     (4,(0.0,-1,4,false))
>     (5,(2.147483647E9,-1,-1,false))
>     (6,(2.147483647E9,-1,-1,false))
>     (7,(2.147483647E9,-1,-1,false))
>     (0,(4.0,1,4,false))
>     (1,(3.0,2,4,false))
>     (2,(2.0,3,4,false))
>     (3,(1.0,4,4,false))
>     (4,(0.0,-1,4,false))
>     (5,(2.147483647E9,-1,-1,true))
>     (6,(2.147483647E9,-1,-1,false))
>     (7,(2.147483647E9,-1,-1,false))
>     Edge(0,1,0)
>     Edge(1,2,0)
>     Edge(2,3,0)
>     Edge(3,4,0)
>     Edge(0,4,0)
>     Edge(5,6,0)
>     Edge(6,7,0)
>     Edge(7,5,0)
>     Edge(3,2,0)
>     Edge(2,3,0)
>     startId: Long = 5
>     goalId: Long = 4
>     INFINITY: Double = 2.147483647E9
>     path: org.apache.spark.graphx.Graph[(Double, Long, Long, Boolean),Int] = org.apache.spark.graphx.impl.GraphImpl@48840ec0
>     defined type alias PN
>     initMsg: Long = 5
>     mergeMsg: (msg1: org.apache.spark.graphx.VertexId, msg2: org.apache.spark.graphx.VertexId)org.apache.spark.graphx.VertexId
>     vprog: (id: org.apache.spark.graphx.VertexId, attr: PN, msg: org.apache.spark.graphx.VertexId)PN
>     sendMsg: (triplet: org.apache.spark.graphx.EdgeTriplet[PN,Int])Iterator[(org.apache.spark.graphx.VertexId, org.apache.spark.graphx.VertexId)]
>     result2: org.apache.spark.graphx.Graph[(Double, Long, Long, Boolean),Int] = org.apache.spark.graphx.impl.GraphImpl@583a0b1e
>     finalResult: org.apache.spark.graphx.Graph[(Double, Long, Long, Boolean),Int] = org.apache.spark.graphx.impl.GraphImpl@3d2d339

In [ ]:

println(finalResult.edges.collect.mkString("\n"))


  

>     Edge(0,1,0)
>     Edge(1,2,0)
>     Edge(2,3,0)
>     Edge(3,4,0)
>     Edge(0,4,0)
>     Edge(5,6,0)
>     Edge(6,7,0)
>     Edge(7,5,0)
>     Edge(3,2,0)
>     Edge(2,3,0)